# Providing context to an LLM using chromaDB

In [6]:
import os
import json
import openai
import chromadb
from chromadb.utils import embedding_functions
from text_wrap import wrap
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Sets paths and constants
DATA_PATH = "data/archive/*"
CHROMA_PATH = "car_review_embeddings"
EMBEDDING_FUNC_NAME = "multi-qa-MiniLM-L6-cos-v1"
COLLECTION_NAME = "car_reviews"

# Set api key
with open("config.json", mode="r") as json_file:
    config_data = json.load(json_file)

openai.api_key = config_data.get("openai-secret-key")



In [7]:
# Set access to database
client = chromadb.PersistentClient(CHROMA_PATH)

# Set which embedding function we should use
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=EMBEDDING_FUNC_NAME
)

# Open stream to database
collection = client.get_collection(
    name=COLLECTION_NAME, embedding_function=embedding_func
)



Now that we've set everythin up, we are going to:
1. Setup a context prompt
2. Put in an example question
3. Retrieve relevant contextual data from ChromaDB based on the question
4. Concat that into a string (that's what the LLMs like), pass it to the LLM as a "system" prompt. This essentially works just like a passing a question, but it's interpreted as data to base the answer off of by the model.
5. Pass the question to the model
6. Get answer based on relevant context provided by ChromaDB

In [8]:
context = """
You are a customer success employee at a large
 car dealership. Use the following car reviews
 to answer questions: {}
"""

question = """
What's the key to great customer satisfaction
 based on detailed positive reviews?
"""

# Get reviews from ChromaDB
good_reviews = collection.query(
    query_texts=[question],
    n_results=10,
    include=["documents"],
    where={"Rating": {"$gte": 3}},
)

# Concat to one string
reviews_str = ",".join(good_reviews["documents"][0])


print(wrap(reviews_str))

 Great value, awesome reliability, I normally dont do reviews, but thought this was worth writing. I
own 3 pickups and am not brand dedicated because there are so many nice trucks. I own 3 different
trucks. I am drawn to looks first then power and interior. I have notice reading reviews that most
want to beat the trucks up, there are no perfect trucks. The Nissan drew me in because it was
different and I dont recall any recalls. I owned a Xterra and it was bulletproof and am hoping for
the same on my Titan. After looking at all brands and I liked them all, I really like the warranty
on my Titan and I got more equipment for the money than the competitors. My first and second tank of
fuel at 70 mpg I got 17.5, I was very pleased and when it get broke in I hope for 20. I find it to
be very quiet and so were all the others on top of the line. I guess I could beat the truck up with
small complaints but sure makes spending my money on it hard to swallow. If the truck last me for 10
years I w

In [9]:
# Get openAI to summarise the data handed to it vi the system role
good_review_summaries = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": context.format(reviews_str)},
        {"role": "user", "content": question},
    ],
    temperature=0,
    n=1,
)


In [10]:
print(good_review_summaries["choices"][0]["message"]["content"])

Based on the detailed positive reviews, the key to great customer satisfaction seems to be a combination of the following factors:

1. Value for money: Customers appreciate getting more equipment and features for their money compared to competitors.

2. Reliability: Customers value cars that are reliable and have a good track record, with no recalls or major issues.

3. Performance: Customers are drawn to cars that have good power and acceleration.

4. Comfort and quietness: Customers appreciate cars that provide a comfortable and quiet driving experience.

5. Fuel efficiency: Customers are pleased with cars that offer excellent fuel efficiency, reducing the need for frequent trips to the gas station.

6. Warranty and coverage: Customers feel more satisfied when their cars come with a comprehensive warranty and coverage, providing them with peace of mind.

7. Longevity: Customers are pleased when their cars are expected to last for a long time, giving them value for their investment.



Lets now give it some bad reviews and find out which of the reviews are the worst.

In [17]:
question = """
Which of these poor reviews has the
 worst implications about our dealership?
 Explain why.
"""

poor_reviews = collection.query(
    query_texts=[question],
    n_results=5,
    include=["documents"],
    # Rating is low
    where={"Rating": {"$lte": 3}},
)

reviews_str = ",".join(poor_reviews["documents"][0])

print(wrap(reviews_str))

 I have been to the dealership four times and I still have unresolved electrical issues, from the
Bluetooth, backup camera, truck wont open, black screen every other trip turning off, clock wont
work, seatbelts NOT WORKING !!!!!!!! which really pisses me off as I have a child in the car. So
many issues in a short period of time it has become the worst vehicle I have owned unfortunately and
to the point where I will drop the car off back at the dealership and purchase something else. I
don't have the time to deal with a vehicle of 53k value as if it were pieced together from spare
parts from a junkyard. I always read reviews before buying vehicles and I have stuck with Nissan &
Infiniti for the past ten years for their workhorse engines as two of my Nissans went to 289k & 377k
miles but after this SUV I am jumping ship as soon as possible., Loved the car when we bought it.
Good features overall but not as good as the Caddy I traded.  Then the nightmare begins.  the car
has been in the s

In [ ]:
poor_review_analysis = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": context.format(reviews_str)},
        {"role": "user", "content": question},
    ],
    temperature=0,
    n=1,
)

In [13]:
print(wrap(poor_review_analysis["choices"][0]["message"]["content"]))

The first review has the worst implications about the dealership. This is because the customer
mentions multiple unresolved electrical issues with their vehicle, including problems with the
Bluetooth, backup camera, trunk, black screen, clock, and seatbelts. These issues are not minor and
are causing frustration and inconvenience for the customer, especially since they have a child in
the car. The customer also expresses their dissatisfaction with the vehicle and states that they
will drop it off at the dealership and purchase something else. This implies that the dealership has
not been able to effectively address and resolve the customer's concerns, leading to a negative
experience and potentially losing a loyal customer.
